In [1]:
import psycopg2
from psycopg2.extensions import register_adapter, AsIs

In [2]:
def select(register,complexType='r',dbname="mimic"):
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    if complexType=='r':
        select_stament="Sample ="+str(register['s_from'])
    else :
        select_stament="Sample BETWEEN "+str(register['s_from'])+" AND "+str(register['s_to'])
    select_stament = "SELECT sample,amp FROM "+complexType+" WHERE record='"+register['record']+"' AND "+select_stament+" LIMIT 1"
    cur.execute(select_stament)
    for row in cur :
        register[complexType+'_s']=row[0]
        register[complexType+'_a']=row[1]
    conn.close()
    return register

In [3]:
def insertRow(row,dbname="mimic") :
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    insert_statement = 'INSERT into rstq (%s) values %s'
    columns = row.keys()
    values = [row[column] for column in columns]
#    print(cur.mogrify(insert_statement, (AsIs(','.join(columns)), tuple(values))))
    cur.execute(insert_statement, (AsIs(','.join(columns)), tuple(values)))
    conn.commit()
    conn.close()

In [4]:
def difference(first,second):
    if first is None or second is None :
        return None
    else :
        return first-second

In [5]:
def readHeartBeats(record,dbname="mimic") :
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    select_stament = "SELECT s_from,s_to FROM heartbeatstemporal WHERE s_to - s_from < 1000 AND record ='"+record+"' "
    cur.execute(select_stament)
    print(record)
    for row in cur :
        register = {'record':record,'s_from':row[0],'s_to':row[1]}
        register = select(register)
        register = select(register,complexType='s')
        register = select(register,complexType='t')
        register = select(register,complexType='q')
        register['qt'] = difference(register.get('q_s'),register.get('t_s'))
        register['ts'] = difference(register.get('t_s'),register.get('s_s'))
        register['sr'] = difference(register.get('s_s'),register.get('r_s'))
        del register['s_from']
        del register['s_to']
        insertRow(register)
    conn.close()
#        selectrow(record[0])

In [ ]:
def distinctRecord(dbname="mimic"):
    conn = psycopg2.connect("dbname="+dbname)
    cur = conn.cursor()
    select_stament = "SELECT DISTINCT r.record FROM r where record not in (select distinct record from rstq) "
    cur.execute(select_stament)
    for record in cur :
        readHeartBeats(record[0])

In [ ]:
distinctRecord()